# OsmGt example

## Imports and prepare input parameters

In [ ]:
from IPython.display import display

from bokeh.plotting import output_notebook

import geopandas as gpd


output_notebook()


location = "Roanne"


## Get POIs

In [ ]:
%%time
from osmrx import Pois

pois_session = Pois()
pois_session.from_location(location)
print(pois_session.data[1])


## Get Roads

In [ ]:
%%time
from osmrx import Roads

vehicle_session = Roads("vehicle")
vehicle_session.from_location(location)

vehicle_session.additional_nodes = pois_session.data
vehicle_session.build_graph()
roads_data = vehicle_session.data

In [ ]:
print(len(roads_data))

In [ ]:
# gpd.GeoDataFrame([f.to_dict() for f in roads_data], geometry="geometry", crs=f"epsg:{4326}").to_file('dataframe.gpkg', driver='GPKG', layer='name')

## Display roads and nodes

In [ ]:
%%time

from bokeh.plotting import show
import geopandas as gpd
from gdf2bokeh import Gdf2Bokeh

map_session = Gdf2Bokeh(
    "My network map",
    width=800,
    height=600,
    background_map_name="CARTODBPOSITRON"
)

map_session.add_layer_from_geodataframe(
    "Roads",
    gpd.GeoDataFrame(roads_data, geometry="geometry", crs=f"epsg:{4326}"),
    from_epsg=4326,
    color="black")

map_session.add_layer_from_geodataframe(
    "POIs",
    gpd.GeoDataFrame(pois_session.data, geometry="geometry", crs=f"epsg:{4326}"),
    from_epsg=4326,
    color="blue", size=9)

map_session.add_layers_on_maps()

show(map_session.figure)


## Check topology details

In [ ]:
%%time

roads_topo_data = vehicle_session.topology_checker()

map_session = Gdf2Bokeh(
    "My topology network map",
    width=800,
    height=600,
    background_map_name="CARTODBPOSITRON"
)

map_session.add_layer_from_dict_list(
    "Roads unchanged",
    roads_topo_data.lines_unchanged,
    from_epsg=4326,
    color="black")

map_session.add_layer_from_dict_list(
    "Roads added",
    roads_topo_data.lines_added,
    from_epsg=4326,
    color="green")

map_session.add_layer_from_dict_list(
    "Roads split",
    roads_topo_data.lines_split,
    from_epsg=4326,
    color="orange")

map_session.add_layer_from_dict_list(
    "Intersections added",
    roads_topo_data.intersections_added,
    from_epsg=4326,
    color="red")

map_session.add_layer_from_dict_list(
    "POIs added",
    roads_topo_data.nodes_added,
    from_epsg=4326,
    color="blue",
    size=9)

map_session.add_layers_on_maps()

show(map_session.figure)


In [2]:
from bokeh.plotting import show
from osmrx import GraphAnalysis
import geopandas as gpd
from gdf2bokeh import Gdf2Bokeh
from shapely import Point
from IPython.display import display

from bokeh.plotting import output_notebook

import geopandas as gpd


output_notebook()
a = Point(4.0793058, 46.0350304)
b = Point(4.0725246, 46.0397676)
nodes = [a, b]
vehicle_session = GraphAnalysis("vehicle", nodes)


paths_found = vehicle_session.get_shortest_path()
paths_found = [f for f in paths_found]
map_session = Gdf2Bokeh(
    "My computed path",
    width=800,
    height=600,
    background_map_name="CARTODBPOSITRON"
)

map_session.add_layer_from_geodataframe(
    "Roads",
    gpd.GeoDataFrame([f for f in vehicle_session.data], geometry="geometry", crs=f"epsg:{4326}"),
    from_epsg=4326,
    color="black")

map_session.add_layer_from_geom_list(
    "Path found",
    [f["geometry"] for f in paths_found[0].features()],
    from_epsg=4326,
    color="blue")

map_session.add_layer_from_geom_list(
    "From",
    [a],
    from_epsg=4326,
    color="red",
    size=9)

map_session.add_layer_from_geom_list(
    "To",
    [b],
    from_epsg=4326,
    color="green",
    size=9)

map_session.add_layers_on_maps()

show(map_session.figure)

Loading BokehJS ...

2023-04-05 00:46:49 - GraphAnalysis   - INFO     : Building OsmFeatureModes.vehicle Data
2023-04-05 00:46:49 - GraphAnalysis   - INFO     : Building the query
2023-04-05 00:46:49 - GraphAnalysis   - INFO     : From 46.030894407456486, 4.068388607456487, 46.04390359254351, 4.083441792543514
2023-04-05 00:46:49 - GraphAnalysis   - INFO     : Building the query
2023-04-05 00:46:49 - GraphAnalysis   - INFO     : Execute the query
2023-04-05 00:46:49 - GraphAnalysis   - INFO     : OverpassApi: Query 200:OK in 0.3 sec.
2023-04-05 00:46:49 - GraphAnalysis   - INFO     : Network cleaning...
2023-04-05 00:46:49 - GraphAnalysis   - INFO     : Starting: Adding new nodes on the network
2023-04-05 00:46:49 - GraphAnalysis   - INFO     : Find nearest line for each node
2023-04-05 00:46:49 - GraphAnalysis   - INFO     : Split line
2023-04-05 00:46:49 - GraphAnalysis   - INFO     : Starting: Find intersections
2023-04-05 00:46:49 - GraphAnalysis   - INFO     : Done: Find intersections
2023-04-05 00:46

In [ ]:
from bokeh.plotting import show
import geopandas as gpd
from gdf2bokeh import Gdf2Bokeh
from shapely import Point

vehicle_session = Roads("pedestrian")

a = [Point(4.0793058, 46.0350304)]
b = [Point(4.0725246, 46.0397676)]

paths_found = vehicle_session.shortest_path(
    a[0],
    b[0]
)
paths_found = [f for f in paths_found]
map_session = Gdf2Bokeh(
    "My computed path",
    width=800,
    height=600,
    background_map_name="CARTODBPOSITRON"
)

map_session.add_layer_from_geom_list(
    "Path found",
    [paths_found[0].path],
    from_epsg=4326,
    color="blue")

map_session.add_layer_from_geom_list(
    "From",
    a,
    from_epsg=4326,
    color="red",
    size=9)

map_session.add_layer_from_geom_list(
    "To",
    b,
    from_epsg=4326,
    color="green",
    size=9)

map_session.add_layers_on_maps()

show(map_session.figure)